# 任务三

In [ ]:
# 导入数据
import pandas as pd
df1 = pd.read_excel(r'D:\研究生资料\玛氏数据\5-22\chengdu_521.xlsx')
df_POI = pd.read_excel(r'D:\研究生资料\玛氏数据\chengdu_poi_category.xlsx')

## 一、	巧克力订单平均下单时间小于6个月的门店对应POI销量的统计

1)	从订单表中筛选出卖巧克力的门店；

In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)

2)	求出卖巧克力门店中平均下单时间小于6个月的门店；
（平均下单时间=（最晚下单时间-最早下单时间）/(订单总量-1)）

In [ ]:
# 把Chocolate的那部分分成下单周期小于6个月和下达周期大于6个月两类
Cstore_id_da6 = [] # 用来储存买Chocolate的那部分下单周期大于6个月的'store_code'
Cstore_id_xiao6 = [] # 用来储存买Chocolate的那部分下单周期小于等于6个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da6.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            Cstore_id_da6.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao6.append(Chocolate_storeid[i]) 

3)	求出（2）中每个门店的poi，然后去重、去充电宝，根据距离由小到大选择前20个POI；

In [ ]:
# Cstore_id_xiao6对应的poi
Cstore_id_xiao6_POI=df_POI[df_POI['storecode'].isin(Cstore_id_xiao6)]
# 去充电宝
Cstore_id_xiao6_POI=Cstore_id_xiao6_POI.drop(Cstore_id_xiao6_POI[Cstore_id_xiao6_POI['s_cat_value']=='充电宝'].index)
# 根据距离由小到大选择前20个POI
CC_da6_POI.sort_values(['storecode','s_cat_id','distance'],ascending=[1,0,1],inplace=True)
k1_1 = CC_da6_POI.groupby(['storecode','s_cat_id']).head(20)

4)	求出（2）中每个门店巧克力的销量（case-ordered）；

In [ ]:
# 在Cstore_id_xiao6里面筛选出卖巧克力的门店；
demo1=df1.loc[df1['store_code'].isin(Cstore_id_xiao6)]
demo2=demo1.loc[demo1['ProductCategory']=='Chocolate']
Cstore_id_xiao6_Cho=list(set(list(demo2['store_code'])))
# 门店巧克力销售加总
cho_sum=[] 
for i in range(len(Cstore_id_xiao6_Cho)):
    demo3 = df1.loc[df1['store_code']==Cstore_id_xiao6_Cho[i]]
    sum_1 = demo3.loc[demo3['ProductCategory']=='Chocolate','case_ordered'].sum()
    cho_sum.append(sum_1)

5)	将（4）中的每个门店销量赋值到（3）中每个门店对应的POI上；

In [ ]:
# k1_1赋上销量信息
# 构建门店编号及其销量信息的表
data3={
    'Cstore_id_xiao6_Cho':Cstore_id_xiao6_Cho,
    'cho_sum':cho_sum
}
data3=pd.DataFrame(data3)
data3.sort_values(by='Cstore_id_xiao6_Cho',inplace=True)

k1_1.sort_values(by='storecode',inplace=True)

sum_2=list(data3['cho_sum'])
s_id =list(data3['Cstore_id_xiao6_Cho'])

sum_3 =[]
for i in range(len(s_id)):
    cc=len(k1_1.loc[k1_1['storecode']==s_id[i]])
    for j in range(cc):
        sum_3.append(sum_2[i])


k1_1['总销量']=sum_3

fff=k1_1[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

6)	求每个POI的销量累计值，按照销量由大到小排序，形成list1；


In [ ]:
k5_4 = fff.groupby(['s_cat_id','s_cat_value'])['总销量'].sum()
k5_4 = k5_4.reset_index()
k5_4.sort_values(['总销量'],ascending=[0],inplace=True)
tt= list(k5_4['s_cat_id'])
r1=[]
r2=[]
r3=[]
r4=[]
for i in range(len(tt)):
    r1.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_id'])))[0])
    r2.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'b_cat_value'])))[0])
    r3.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_id'])))[0])
    r4.append(list(set(list(fff.loc[fff['s_cat_id']==tt[i],'m_cat_value'])))[0])
k5_4['b_cat_id']=r1
k5_4['b_cat_value']=r2
k5_4['m_cat_id']=r3
k5_4['m_cat_value']=r4

k5_4=k5_4[['b_cat_id','b_cat_value','m_cat_id','m_cat_value','s_cat_id','s_cat_value','总销量']]

7)	选取销量的前20组成list1-1

In [ ]:
k5_5=k5_4.head(20)
k5_5.to_excel(r'C:\Users\Administrator\Desktop\任务三（一）.xlsx')

## 二、	巧克力订单平均时间大于6个月和不销售巧克力的门店对用POI出现次数统计

1)	求不卖巧克力和巧克力平均下单时间超过6个月的门店



In [ ]:
# 卖Chocolate的'store_code'
Chocolate_storeid=set(df1.loc[df1['ProductCategory']=='Chocolate','store_code'])
Chocolate_storeid=list(Chocolate_storeid)
# 不卖Chocolate的'store_code'
all_storeid = list(df1['store_code'])
NoChocolate_storeid=list(set(all_storeid)-set(Chocolate_storeid))

# 把Chocolate的那部分分成下单周期小于6个月和下达周期大于6个月两类
Cstore_id_da6 = [] # 用来储存买Chocolate的那部分下单周期大于6个月的'store_code'
Cstore_id_xiao6 = [] # 用来储存买Chocolate的那部分下单周期小于等于6个月的'store_code'
for i in range(len(Chocolate_storeid)):
    demo0 = df1.loc[df1['store_code']==Chocolate_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chocolate的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chocolate']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da6.append(Chocolate_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            Cstore_id_da6.append(Chocolate_storeid[i])
        else:
            Cstore_id_xiao6.append(Chocolate_storeid[i]) 
             
# NoChocolate_storeid 与 Cstore_id_da6的并集
No_da6_id = list(set(NoChocolate_storeid)|set(Cstore_id_da6))

2)	求出（1）中门店对应的POI；


In [ ]:
# No_da6_id 对应的POI
No_da6_id_POI=df_POI[df_POI['storecode'].isin(No_da6_id)]

3)	统计（2）中每个POI出现的次数（去充电宝），并按出现次数排序，形成list2


In [ ]:
# 去充电宝
No_da6_id_POI=No_da6_id_POI.drop(No_da6_id_POI[No_da6_id_POI['s_cat_value']=='充电宝'].index)
# 在No_da6_id_POI的基础上，求POI出现次数
k1_1 = No_da6_id_POI
k1_1['nums']=0
uu_1=list(k1_1.groupby(['s_cat_id']).count()['nums'].index) # 表示s_cat_id
uu_2=list(k1_1.groupby(['s_cat_id']).count()['nums'].values) # 表示出现次数
uu_3=[]
uu_4=[]
uu_5=[]
uu_6=[]
uu_7=[]
# 'b_cat_id','b_cat_value','m_cat_id','m_cat_value'
for i in range(len(uu_1)):
    f1=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'b_cat_id'])[0]
    f2=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'b_cat_value'])[0]
    f3=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'m_cat_id'])[0]
    f4=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'m_cat_value'])[0]
    f5=list(k1_1.loc[k1_1['s_cat_id']==uu_1[i],'s_cat_value'])[0]
    uu_3.append(f1)
    uu_4.append(f2)
    uu_5.append(f3)
    uu_6.append(f4)
    uu_7.append(f5)

data3={
    'b_cat_id':uu_3,
    'b_cat_value':uu_4,
    'm_cat_id':uu_5,
    'm_cat_value':uu_6,
    's_cat_id':uu_1,
    's_cat_value':uu_7,
    '出现次数':uu_2
    
}
k1_2=pd.DataFrame(data3)
# 降序
k1_3=k1_2.sort_values(by='出现次数',ascending=False)

4)	选取出现次数的前20组成list2-1；

In [ ]:
k1_4=k1_3.head(20)
k1_4.to_excel(r'C:\Users\Administrator\Desktop\任务三（二）.xlsx')

## 三、	分析对比

1)	求list1-1 减去（list1-1和list2-1 的交集），其实就是求list1-1与list2-1的差集

In [ ]:
# list1-1(k5_5)
# list2-1(k1_4)
b1=set(k5_5['s_cat_id'])
b2=set(k1_4['s_cat_id'])
b12 = list(set(b1)-set(b2))
d13_df = k5_5[k5_5['s_cat_id'].isin(b12)]
d13_df.to_excel(r'C:\Users\Administrator\任务三（三）.xlsx')